In [12]:
import os
import os
import numpy as np

from IPython.display import Video


def get_label(ans):
    try:
        ans = int(ans)
    except:
        print("Wrong Input")
        return False
    if ans not in [1,2,3]:
        print("Invalid option.")
        return False
    if ans == 1:
        return [1, 0]
    elif ans == 2:
        return [0, 1]
    else:
        return [0.5, 0.5]


def create_human_label(save_dir, env_name, num_query=1000, start_idx=None, width=1000, height=500):
    video_path = os.path.join(save_dir, env_name)
    os.makedirs(os.path.join(video_path, "label"), exist_ok=True)
    print("START!")
    if start_idx:
        assert start_idx > 0, "you must input with video number (1, 2, 3, ...)"
        interval = range(start_idx - 1, num_query)
    else:
        interval = range(num_query)
        
    for i in interval:
        label = False
        while not label:
            print(f"\nVideo {i + 1}")
            video_file = os.path.join(video_path, f"idx{i}.mp4")
            display(Video(video_file, width=width, height=height, html_attributes="loop autoplay"))
            reward = input(f"[{i + 1}/{num_query}] Put Preference (1 (left), 2 (right), 3 (equal)):  ").strip()
            label = get_label(reward)
            if label:
                with open(os.path.join(video_path, "label", f"label_{i}.txt"), "w") as f:
                    f.write(reward)

In [ ]:
### create human label in save_dir, you could fix the start point.
create_human_label(save_dir="../video", env_name="antmaze-large-diverse-v2", start_idx=956, num_query=1000)

In [17]:
import os
import glob
import pickle
import numpy as np
from tqdm import trange

# make final pickle file from separated label files.
def merge_labels(save_dir, env_name="antmaze-medium-play-v2", num_query=1000, query_len=100, seed=3407):
    label_dir = os.path.join(save_dir, env_name, "label")
    # label_files = sorted(glob.glob(os.path.join(label_dir, "*.txt")), key=lambda x: int(x.split(".")[0].split("_")[-1]))
    labels = []
    for idx in trange(num_query):
        assert os.path.exists(os.path.join(label_dir, f"label_{idx}.txt")), f"labeling is not finished. {idx + 1} / {num_query}"
        with open(os.path.join(label_dir, f"label_{idx}.txt")) as f:
            choice = int(f.read().strip())
            if choice == 1:
                _label = 0
            elif choice == 2:
                _label = 1
            elif choice == 3:
                _label = -1
        labels.append(_label)
        
    # labels = np.array(labels)
        
    with open(os.path.join(save_dir, env_name, f"human_labels_numq{num_query}_len{query_len}_s{seed}.pkl"), "wb") as f:
        pickle.dump(labels, f)

In [39]:
merge_labels(save_dir="../video", env_name="antmaze-medium-play-v2")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 22433.63it/s]


In [40]:
merge_labels(save_dir="../video", env_name="antmaze-medium-diverse-v2")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 19003.26it/s]


In [18]:
merge_labels(save_dir="../video", env_name="antmaze-large-diverse-v2")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 20405.77it/s]


In [36]:
with open("../video/antmaze-medium-play-v2/human_labels_numq1000_len100_s3407.pkl", "rb") as f:
    labels = pickle.load(f)